In [1]:
import sys
from collections import defaultdict
import multiprocessing
from multiprocessing import Pool
from functools import partial
import timeit

import ribopy
from ribopy import Ribo
from ribopy.core.get_gadgets import get_region_boundaries, get_reference_names

import numpy as np
import pandas as pd

sys.path.insert(0, '../snp')
from ref_lib.Fasta import FastaEntry, FastaFile

In [2]:
mouse_transcriptome_file = "../../mouse_itp_reference/transcriptome/varnt_masked_and_filtered_mouse_transcriptome.fa.gz"
human_transcriptome_file = "../../../itp/human_itp_reference/transcriptome/appris_human_v2_after_filtering.fa.gz"

mouse_ribo_file = "../../mouse-itp_v5.ribo"
human_ribo_file = "../../../itp/human-itp_v4.ribo"

LEN_MIN = 29
LEN_MAX = 35

In [3]:
mouse_sequences = dict()

with FastaFile(mouse_transcriptome_file) as mouse_transcriptome:
    for entry in mouse_transcriptome:
        mouse_sequences[ entry.header ] = entry.sequence



In [4]:
human_sequences = dict()

with FastaFile(human_transcriptome_file) as human_transcriptome:
    for entry in human_transcriptome:
        human_sequences[ entry.header ] = entry.sequence


In [5]:
mouse_ribo = Ribo(mouse_ribo_file)

In [6]:
human_ribo = Ribo(human_ribo_file)

In [7]:
mouse_region_boundaries = get_region_boundaries(mouse_ribo._handle)
mouse_gene_names        = get_reference_names(mouse_ribo._handle)

mouse_ref_name_and_boundaries = zip(mouse_gene_names, mouse_region_boundaries)

mouse_cds_boundaries = dict()

for e in list(mouse_ref_name_and_boundaries):
    mouse_cds_boundaries[e[0]] = e[1][1]

In [8]:
human_region_boundaries = get_region_boundaries(human_ribo._handle)
human_gene_names        = get_reference_names(human_ribo._handle)

human_ref_name_and_boundaries = zip(human_gene_names, human_region_boundaries)

human_cds_boundaries = dict()

for e in list(human_ref_name_and_boundaries):
    human_cds_boundaries[e[0]] = e[1][1]

In [9]:
mouse_ribo = Ribo(mouse_ribo_file)

In [10]:
def get_coverage_at_length(this_ribo, experiment, rpf_length):
    return this_ribo.get_coverage(experiment  = experiment,
                                  range_lower = rpf_length, 
                                  range_upper = rpf_length)

def make_coverage_dict_of_experiment(this_ribo, experiment, len_min, len_max):
    result_dict = defaultdict(dict)
    
    for i in range(len_min, len_max + 1):
        result_dict[i] = get_coverage_at_length(this_ribo, experiment, i)
        
    return result_dict

In [11]:
MII_1_coverages = make_coverage_dict_of_experiment(mouse_ribo, 
                                                   experiment = "20210301-ITP-MII-25-B", 
                                                   len_min    = LEN_MIN, 
                                                   len_max    = LEN_MAX )

In [12]:
list(MII_1_coverages.keys())

[29, 30, 31, 32, 33, 34, 35]

In [13]:
def init_frame_triplet():
    return [0, 0, 0]

def count_nucleotides(coverage_dict, 
                      cds_annotation_dict, 
                      sequence_dict, 
                      rpf_len,
                      left_span = 2, right_span = 1 ):
    """
    Given a footprint length, this functio creates a table where each row
    comes from nucleotide sequences
    and the three columns correspond to the frames 0,1,2
    
    coverage_dict:
       gene_identifier -> coverage vector
       
    cds_annotation_dict:
       gene_identifier -> [cds_start, cds_end]
       
    sequence_dict:
       gene_identifier -> gene_sequence
       
    rpf_len:
       read length, ribosome protected footpront length
       
    left_span:  the last nucleotides before the 3' end of the read 
    
    right_span: the first nucleotides  the 3' end of the read
    
    """
    
    nuc_counts = defaultdict(init_frame_triplet)
    
    
    for gene, coverage in coverage_dict.items():
        for position in range( cds_annotation_dict[gene][0], cds_annotation_dict[gene][1] ):
            if coverage[position] == 0:
                continue
                
            this_sequence     = sequence_dict[gene][position + rpf_len - left_span: \
                                                    position + rpf_len + right_span]
            
            if "N" in this_sequence:
                continue
            
            relative_position = position - cds_annotation_dict[gene][0]
            
            this_frame = relative_position % 3
            
            nuc_counts[this_sequence][this_frame] += coverage[position]
        
    return nuc_counts

In [52]:
def count_nucleotides_parallel( coverage_dict, 
                                cds_annotation_dict, 
                                sequence_dict, 
                                rpf_min, 
                                rpf_max,
                                processes = 6,
                                left_span = 2, right_span = 1 ):
    
    """
    Wrapper for the function `count_nucleotides`.
    
    
    It takes a range of ribosome protected footprint lengths and runs count_nucletides
    function on these individual lengths.
    
    """
    
    rpf_range = list(range(rpf_min, rpf_max + 1))
    
    # Without this global, we have an error at the Pool.map step.
    global f
    
    def f(x):
       return count_nucleotides( coverage_dict[x], cds_annotation_dict, sequence_dict, rpf_len = x,
                      left_span = left_span, right_span = right_span ) 
    
    with Pool(processes) as p:
        mapped_list = p.map(f, rpf_range)
        
    return(mapped_list)
        


In [56]:
MII_1_counts = count_nucleotides_parallel( 
                                coverage_dict = MII_1_coverages, 
                                cds_annotation_dict = mouse_cds_boundaries, 
                                sequence_dict = mouse_sequences, 
                                rpf_min = LEN_MIN, 
                                rpf_max = LEN_MAX,
                                processes = 4,
                                left_span = 2, right_span = 1 )

In [51]:
MII_1_counts[1]['AGA']

[176, 131, 60]

In [48]:
def max_by_cyclic_shift(df):
    """
    Bring maximal counts in the frames to the frame 0 for each length.
    This is done via a cyclic shift so that the maximal is at frame 0.
    """
    adjusted_tuples = []

    for r,v in df.iterrows():
        max_index      = np.argmax(v)
        adjusted_tuple = [v[ (i + max_index) % 3] for i in range(3)  ]
        adjusted_tuples.append(adjusted_tuple)

    mydf = pd.DataFrame(adjusted_tuples)
    
    return mydf
    
    
def _adjust_p_sites(nucleotide_counts_list):
    """
    Reports a dataframe where frames are reported per length
    """
    summed_frames_per_length = []
    
    
    # Sum the values for the frames 0,1,2
    # collect them in a dataframe df_s
    for nuc_counts in nucleotide_counts_list:
        df = pd.DataFrame(nuc_counts)
        summed_frames_per_length.append( df.sum(axis=1) )
        
    summed_frames_df       = pd.DataFrame(summed_frames_per_length)
    p_site_adjusted_frames = max_by_cyclic_shift(summed_frames_df)
    
    return p_site_adjusted_frames

def adjust_p_sites(nucleotide_counts_list):
    adjusted_df = _adjust_p_sites(nucleotide_counts_list)
    
    return adjusted_df.sum(axis=0)

def calculate_periodicity_percentage(npseries):
    
    return 100 * ( npseries / np.sum(npseries) )


def _adjust_frames_by_sequence_indiv_length(nucleotide_counts_df):
    adjusted_frames = max_by_cyclic_shift(nucleotide_counts_df)
    
    return adjusted_frames.sum(axis = 0)

def adjust_frames_by_sequence(nucleotide_counts_list):
    result = [0, 0, 0]
    
    for current_len_data in nucleotide_counts_list:
        this_df            = pd.DataFrame(current_len_data).transpose()
        this_adjusted_list = _adjust_frames_by_sequence_indiv_length(this_df)
        result += this_adjusted_list
        
    return (result)



In [20]:
sample_df_1 = pd.DataFrame(MII_1_counts[1]).transpose()

In [46]:
a = adjust_frames_by_sequence(MII_1_counts)
calculate_periodicity_percentage(a)

0    46.670737
1    26.450915
2    26.878348
dtype: float64

In [23]:
a = _p_site_adjust(sample_df_1)

In [24]:
calculate_periodicity_percentage(a)

0    47.200822
1    25.719055
2    27.080123
dtype: float64

In [50]:
b = adjust_p_sites(MII_1_counts)
calculate_periodicity_percentage(b)

0    37.489245
1    32.332288
2    30.178467
dtype: float64

In [57]:
experiments = mouse_ribo.experiments

nucleotide_counts_2_1 = dict()
nucleotide_counts_0_1 = dict()

for e in experiments:
    print(e)
    this_coverage = make_coverage_dict_of_experiment(
                            mouse_ribo, 
                            experiment = e, 
                            len_min    = LEN_MIN, 
                            len_max    = LEN_MAX )
    
    nucleotide_counts_2_1[e] = count_nucleotides_parallel( 
                                    coverage_dict       = this_coverage, 
                                    cds_annotation_dict = mouse_cds_boundaries ,
                                    sequence_dict       = mouse_sequences, 
                                    rpf_min = LEN_MIN, rpf_max = LEN_MAX,
                                    left_span           = 2, 
                                    right_span          = 1 )
    
    nucleotide_counts_0_1[e] = count_nucleotides_parallel( 
                                    coverage_dict       = this_coverage, 
                                    cds_annotation_dict = mouse_cds_boundaries ,
                                    sequence_dict       = mouse_sequences, 
                                    rpf_min = LEN_MIN, rpf_max = LEN_MAX,
                                    left_span           = 0, 
                                    right_span          = 1 )   


20210301-ITP-MII-25-B
20210301-ITP-MII-50-A
20210301-ITP-MII-50-B
20210318-ITP-MII-50-B
20210513-ITP-1cell-cross-50-A
20210513-ITP-1cell-cross-50-B
20210513-ITP-1cell-cross-50-C
20210513-ITP-1cell-cross-50-D
20210513-ITP-1cell-cross-50-E
20210513-ITP-2cell-cross-50-B
20210513-ITP-2cell-cross-50-C
20210513-ITP-2cell-cross-50-F
20210513-ITP-4cell-cross-50-B
20210513-ITP-4cell-cross-50-C
20210513-ITP-4cell-cross-50-D
20210513-ITP-8cell-cross-50-A
20210513-ITP-8cell-cross-50-B
20210513-ITP-8cell-cross-50-C
20210513-ITP-8cell-cross-50-D
20210614-ITP-GV-50-A
20210614-ITP-GV-50-B
20210614-ITP-GV-50-C
20210614-ITP-GV-50-E
20210614-ITP-GV-50-F
20210614-ITP-MII-50-D


In [75]:
this_adjusted_frames = adjust_frames_by_sequence(nucleotide_counts_2_1[mouse_ribo.experiments[3]]  )
calculate_periodicity_percentage(this_adjusted_frames)

0    46.120797
1    26.120520
2    27.758683
dtype: float64

In [77]:
this_adjusted_frames = adjust_p_sites(nucleotide_counts_2_1[mouse_ribo.experiments[3]]  )
calculate_periodicity_percentage(this_adjusted_frames)

0    37.019068
1    32.112288
2    30.868644
dtype: float64

In [ ]:
20210301-ITP-MII-50-B

In [80]:
mouse_adjusted_frame_percentages = dict()
mouse_raw_frame_percentages      = dict()

for e in mouse_ribo.experiments:
    adjusted_frames            = adjust_frames_by_sequence(nucleotide_counts_2_1[e]  )
    p_raw_frames               = adjust_p_sites(nucleotide_counts_2_1[e])
    
    mouse_raw_frame_percentages[e]      = calculate_periodicity_percentage(p_raw_frames)
    mouse_adjusted_frame_percentages[e] = calculate_periodicity_percentage(adjusted_frames)    

In [86]:
mouse_adjusted_frame_percentages_df = pd.DataFrame(mouse_adjusted_frame_percentages).transpose()
mouse_raw_frame_percentages_df = pd.DataFrame(mouse_raw_frame_percentages).transpose()

In [89]:
mouse_adjusted_frame_percentages_df.to_csv("mouse_adjusted_frame_percentages.csv")

In [ ]:
mouse_raw_frame_percentages_df

In [90]:
mouse_raw_frame_percentages_df.to_csv("mouse_raw_frame_percentages.csv")

In [88]:
mouse_raw_frame_percentages_df

,0,1,2
20210301-ITP-MII-25-B,37.489245,32.332288,30.178467
20210301-ITP-MII-50-A,37.019068,32.112288,30.868644
20210301-ITP-MII-50-B,36.911532,30.491706,32.596761
20210318-ITP-MII-50-B,36.300136,31.971893,31.727971
20210513-ITP-1cell-cross-50-A,37.168580,30.238705,32.592715
20210513-ITP-1cell-cross-50-B,36.218600,31.001484,32.779916
20210513-ITP-1cell-cross-50-C,37.133682,32.524691,30.341627
20210513-ITP-1cell-cross-50-D,36.679381,32.248149,31.072470
20210513-ITP-1cell-cross-50-E,36.819110,31.516946,31.663944
20210513-ITP-2cell-cross-50-B,37.605734,30.435105,31.959161


--------------------------------------------------------

## HUMAN DATA

In [91]:
human_ribo.experiments

('20191203-Kit-10M-Monosome-1',
 '20191203-Kit-10M-Monosome-2',
 '20191203-Kit-10M-Monosome-3',
 '20201104-ITP-100-5mM-50-1',
 '20201209-ITP-100-5mM-6',
 '20210131-ITP-100-5mM-50_diluted-1')

In [92]:
human_experiments = human_ribo.experiments


human_nucleotide_counts_2_1 = dict()


for e in human_experiments:
    print(e)
    this_coverage = make_coverage_dict_of_experiment(
                            human_ribo, 
                            experiment = e, 
                            len_min    = LEN_MIN, 
                            len_max    = LEN_MAX )
    
    
    human_nucleotide_counts_2_1[e] = count_nucleotides_parallel( 
                                    coverage_dict       = this_coverage, 
                                    cds_annotation_dict = human_cds_boundaries ,
                                    sequence_dict       = human_sequences, 
                                    rpf_min = LEN_MIN, rpf_max = LEN_MAX,
                                    left_span           = 2, 
                                    right_span          = 1 )   



20191203-Kit-10M-Monosome-1
20191203-Kit-10M-Monosome-2
20191203-Kit-10M-Monosome-3
20201104-ITP-100-5mM-50-1
20201209-ITP-100-5mM-6
20210131-ITP-100-5mM-50_diluted-1


In [96]:
human_adjusted_frame_percentages = dict()
human_raw_frame_percentages      = dict()

for e in human_ribo.experiments:
    adjusted_frames            = adjust_frames_by_sequence(human_nucleotide_counts_2_1[e]  )
    p_raw_frames               = adjust_p_sites(human_nucleotide_counts_2_1[e])
    
    human_raw_frame_percentages[e]      = calculate_periodicity_percentage(p_raw_frames)
    human_adjusted_frame_percentages[e] = calculate_periodicity_percentage(adjusted_frames) 

In [97]:
human_raw_frame_percentages_df = pd.DataFrame(human_raw_frame_percentages).transpose()
human_raw_frame_percentages_df.to_csv("human_raw_frame_percentages.csv")
human_raw_frame_percentages_df

,0,1,2
20191203-Kit-10M-Monosome-1,39.006724,31.846913,29.146363
20191203-Kit-10M-Monosome-2,38.693376,32.024315,29.282310
20191203-Kit-10M-Monosome-3,38.623567,32.480144,28.896289
20201104-ITP-100-5mM-50-1,38.641237,32.225242,29.133521
20201209-ITP-100-5mM-6,37.139046,31.723705,31.137249
20210131-ITP-100-5mM-50_diluted-1,37.707792,31.300839,30.991369


In [98]:
human_adjusted_frame_percentages_df = pd.DataFrame(human_adjusted_frame_percentages).transpose()
human_adjusted_frame_percentages_df.to_csv("human_adjusted_frame_percentages.csv")
human_adjusted_frame_percentages_df

,0,1,2
20191203-Kit-10M-Monosome-1,47.847891,25.507283,26.644826
20191203-Kit-10M-Monosome-2,47.466395,25.968606,26.564998
20191203-Kit-10M-Monosome-3,47.432668,25.929824,26.637508
20201104-ITP-100-5mM-50-1,48.485020,25.323344,26.191635
20201209-ITP-100-5mM-6,48.398547,25.023012,26.578441
20210131-ITP-100-5mM-50_diluted-1,48.164119,24.983320,26.852562
